# Week 10 Lab: Building ML APIs with FastAPI

**CS 203: Software Tools and Techniques for AI**

---

## Lab Overview

In this lab, you will learn to:
1. **Build REST APIs** with FastAPI from scratch
2. **Validate inputs** using Pydantic models
3. **Serve ML models** via HTTP endpoints
4. **Handle errors** gracefully
5. **Test APIs** with automated tests

**Goal**: Create a production-ready ML prediction API.

---

## Setup

In [ ]:
# Install required packages
!pip install "fastapi[standard]" scikit-learn joblib pytest httpx uvicorn

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
import os

print("All imports successful!")

---

# Part 1: Understanding REST APIs

Before building APIs, let's understand how they work.

## 1.1 What is a REST API?

```
┌─────────────┐    HTTP Request     ┌─────────────┐
│   Client    │ ─────────────────► │   Server    │
│ (Browser,   │                     │  (FastAPI)  │
│  Python,    │ ◄───────────────── │             │
│  Mobile)    │    HTTP Response    │             │
└─────────────┘                     └─────────────┘
```

**REST** = Representational State Transfer

**Key concepts**:
- **Resources**: Things you can access (e.g., `/users`, `/movies`, `/predictions`)
- **HTTP Methods**: Actions on resources (GET, POST, PUT, DELETE)
- **JSON**: Standard format for data exchange

### Question 1.1 (Solved): Explore a Public API

Let's call a public API to understand the request/response cycle.

In [ ]:
# SOLVED EXAMPLE
import requests

# Call a public API
response = requests.get("https://jsonplaceholder.typicode.com/posts/1")

print(f"Status Code: {response.status_code}")
print(f"Content-Type: {response.headers['Content-Type']}")
print(f"\nResponse Body:")
print(json.dumps(response.json(), indent=2))

### Question 1.2: HTTP Methods

Match each HTTP method to its purpose:

| Method | Purpose |
|--------|----------|
| GET | ? |
| POST | ? |
| PUT | ? |
| DELETE | ? |

Options: Create new resource, Read/retrieve data, Update existing resource, Remove resource

In [ ]:
# YOUR ANSWERS HERE
http_methods = {
    "GET": None,     # Replace None with purpose
    "POST": None,
    "PUT": None,
    "DELETE": None
}

print(http_methods)

### Question 1.3: Make a POST Request

Create a new post using the JSONPlaceholder API.

**Endpoint**: `https://jsonplaceholder.typicode.com/posts`

**Data to send**:
```python
{
    "title": "My First Post",
    "body": "This is the content of my post",
    "userId": 1
}
```

Print the status code and response body.

In [ ]:
# YOUR CODE HERE


---

# Part 2: Train and Save an ML Model

Before serving a model via API, we need to train and save it.

## 2.1 Train a Simple Model

### Question 2.1 (Solved): Train Iris Classifier

In [ ]:
# SOLVED EXAMPLE
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

# Load data
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
accuracy = model.score(X_test, y_test)
print(f"Model accuracy: {accuracy:.2%}")

# Save model
joblib.dump(model, "iris_model.pkl")
print("Model saved to iris_model.pkl")

# Show feature names and target names
print(f"\nFeatures: {iris.feature_names}")
print(f"Classes: {iris.target_names}")

### Question 2.2: Test the Saved Model

Load the saved model and make a prediction for:
- sepal_length: 5.1
- sepal_width: 3.5
- petal_length: 1.4
- petal_width: 0.2

Print the predicted class name and probabilities.

In [ ]:
# YOUR CODE HERE


---

# Part 3: Build Your First FastAPI

Now let's create an API to serve predictions.

## 3.1 Creating the API

We'll write FastAPI code to files and run them.

### Question 3.1 (Solved): Hello World API

Create a simple FastAPI application.

In [ ]:
# SOLVED EXAMPLE
# Write the API code to a file

api_code = '''
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, FastAPI!"}

@app.get("/hello/{name}")
def greet(name: str):
    return {"greeting": f"Hello, {name}!"}
'''

with open("main_hello.py", "w") as f:
    f.write(api_code)

print("Created main_hello.py")
print("\nTo run: fastapi dev main_hello.py")
print("Then visit: http://127.0.0.1:8000/docs")

### Question 3.2: Add Query Parameters

Modify the API to add an endpoint `/search` that accepts:
- `keyword` (required string)
- `limit` (optional integer, default 10)

Return: `{"keyword": keyword, "limit": limit}`

In [ ]:
# YOUR CODE HERE
# Write the updated API code to a file

api_code = '''
from fastapi import FastAPI
from typing import Optional

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, FastAPI!"}

# ADD YOUR /search ENDPOINT HERE

'''

with open("main_search.py", "w") as f:
    f.write(api_code)

print("Created main_search.py")

---

# Part 4: Input Validation with Pydantic

Pydantic models validate incoming data automatically.

## 4.1 Creating Request Models

### Question 4.1 (Solved): Pydantic Model for Predictions

In [ ]:
# SOLVED EXAMPLE
from pydantic import BaseModel, Field
from typing import Optional, List

class PredictionInput(BaseModel):
    """Input schema for iris prediction."""
    sepal_length: float = Field(..., gt=0, lt=10, description="Sepal length in cm")
    sepal_width: float = Field(..., gt=0, lt=10, description="Sepal width in cm")
    petal_length: float = Field(..., gt=0, lt=10, description="Petal length in cm")
    petal_width: float = Field(..., gt=0, lt=10, description="Petal width in cm")
    
    class Config:
        json_schema_extra = {
            "example": {
                "sepal_length": 5.1,
                "sepal_width": 3.5,
                "petal_length": 1.4,
                "petal_width": 0.2
            }
        }

class PredictionOutput(BaseModel):
    """Output schema for iris prediction."""
    species: str
    confidence: float
    probabilities: List[float]

# Test the model
valid_input = PredictionInput(
    sepal_length=5.1,
    sepal_width=3.5,
    petal_length=1.4,
    petal_width=0.2
)
print(f"Valid input: {valid_input}")

# Test validation error
try:
    invalid_input = PredictionInput(
        sepal_length=-1.0,  # Invalid: negative
        sepal_width=3.5,
        petal_length=1.4,
        petal_width=0.2
    )
except Exception as e:
    print(f"\nValidation error: {e}")

### Question 4.2: Create a Batch Prediction Model

Create a Pydantic model `BatchPredictionInput` that accepts a list of `PredictionInput` items.

Test it with 3 sample flowers.

In [ ]:
# YOUR CODE HERE


---

# Part 5: Complete ML Prediction API

Now let's put it all together.

## 5.1 The Complete API

### Question 5.1 (Solved): Full ML API

In [ ]:
# SOLVED EXAMPLE
# Write the complete ML API to a file

ml_api_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from typing import List, Optional
import joblib
from pathlib import Path

# Create FastAPI app
app = FastAPI(
    title="Iris Species Prediction API",
    description="Predict Iris flower species from measurements",
    version="1.0.0"
)

# Global model variable
model = None
SPECIES_NAMES = ["setosa", "versicolor", "virginica"]

# Pydantic models
class PredictionInput(BaseModel):
    sepal_length: float = Field(..., gt=0, lt=10)
    sepal_width: float = Field(..., gt=0, lt=10)
    petal_length: float = Field(..., gt=0, lt=10)
    petal_width: float = Field(..., gt=0, lt=10)
    
    class Config:
        json_schema_extra = {
            "example": {
                "sepal_length": 5.1,
                "sepal_width": 3.5,
                "petal_length": 1.4,
                "petal_width": 0.2
            }
        }

class PredictionOutput(BaseModel):
    species: str
    confidence: float
    probabilities: List[float]

# Startup event
@app.on_event("startup")
def load_model():
    global model
    model_path = Path("iris_model.pkl")
    if model_path.exists():
        model = joblib.load(model_path)
        print("Model loaded successfully!")
    else:
        print("Warning: Model file not found!")

# Health check
@app.get("/health")
def health_check():
    return {
        "status": "healthy",
        "model_loaded": model is not None
    }

# Prediction endpoint
@app.post("/predict", response_model=PredictionOutput)
def predict(input_data: PredictionInput):
    if model is None:
        raise HTTPException(status_code=503, detail="Model not loaded")
    
    # Prepare features
    features = [[
        input_data.sepal_length,
        input_data.sepal_width,
        input_data.petal_length,
        input_data.petal_width
    ]]
    
    # Predict
    prediction = model.predict(features)[0]
    probabilities = model.predict_proba(features)[0]
    
    return {
        "species": SPECIES_NAMES[prediction],
        "confidence": float(max(probabilities)),
        "probabilities": probabilities.tolist()
    }
'''

with open("ml_api.py", "w") as f:
    f.write(ml_api_code)

print("Created ml_api.py")
print("\nTo run: fastapi dev ml_api.py")
print("Then visit: http://127.0.0.1:8000/docs")

### Question 5.2: Add Batch Prediction Endpoint

Add a `/predict/batch` endpoint to the API that:
1. Accepts a list of prediction inputs
2. Returns a list of predictions
3. Includes a count of predictions made

In [ ]:
# YOUR CODE HERE
# Add the batch endpoint code


---

# Part 6: Error Handling

Production APIs need robust error handling.

## 6.1 Handling Common Errors

### Question 6.1 (Solved): Error Handling Patterns

In [ ]:
# SOLVED EXAMPLE
# Common error handling patterns

error_handling_code = '''
from fastapi import FastAPI, HTTPException
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

@app.post("/predict")
def predict(input_data: PredictionInput):
    try:
        # Check model is loaded
        if model is None:
            logger.error("Prediction failed: Model not loaded")
            raise HTTPException(
                status_code=503,
                detail="Model not available. Please try again later."
            )
        
        # Log the request
        logger.info(f"Prediction request: {input_data.dict()}")
        
        # Make prediction
        features = [[...]]  # Prepare features
        prediction = model.predict(features)[0]
        
        return {"species": SPECIES_NAMES[prediction]}
        
    except HTTPException:
        raise  # Re-raise HTTP exceptions
    except Exception as e:
        logger.error(f"Prediction error: {e}", exc_info=True)
        raise HTTPException(
            status_code=500,
            detail="Internal server error during prediction"
        )
'''

print("Error handling patterns:")
print("1. Log all errors")
print("2. Return appropriate status codes (503 for unavailable, 500 for errors)")
print("3. Never expose internal error details to clients")
print("4. Use try-except to catch unexpected errors")

### Question 6.2: Match Status Codes to Scenarios

Match each scenario to the correct HTTP status code:

| Scenario | Status Code |
|----------|-------------|
| Model loaded, prediction successful | ? |
| Invalid input (negative values) | ? |
| Model file not found | ? |
| Server crashed during prediction | ? |

Options: 200, 422, 500, 503

In [ ]:
# YOUR ANSWERS HERE
scenarios = {
    "prediction_successful": None,
    "invalid_input": None,
    "model_not_found": None,
    "server_crashed": None
}

print(scenarios)

---

# Part 7: Testing Your API

Automated tests verify your API works correctly.

## 7.1 Writing API Tests

### Question 7.1 (Solved): Test with TestClient

In [ ]:
# SOLVED EXAMPLE
# Write tests to a file

test_code = '''
from fastapi.testclient import TestClient
from ml_api import app

client = TestClient(app)

def test_health_check():
    response = client.get("/health")
    assert response.status_code == 200
    data = response.json()
    assert "status" in data
    assert "model_loaded" in data

def test_predict_setosa():
    """Test prediction for setosa species."""
    payload = {
        "sepal_length": 5.1,
        "sepal_width": 3.5,
        "petal_length": 1.4,
        "petal_width": 0.2
    }
    response = client.post("/predict", json=payload)
    assert response.status_code == 200
    data = response.json()
    assert "species" in data
    assert "confidence" in data
    assert data["species"] == "setosa"

def test_predict_invalid_input():
    """Test that invalid input returns 422."""
    payload = {
        "sepal_length": -1.0,  # Invalid
        "sepal_width": 3.5,
        "petal_length": 1.4,
        "petal_width": 0.2
    }
    response = client.post("/predict", json=payload)
    assert response.status_code == 422
'''

with open("test_api.py", "w") as f:
    f.write(test_code)

print("Created test_api.py")
print("\nTo run tests: pytest test_api.py -v")

### Question 7.2: Write Additional Tests

Add tests for:
1. Missing required field (should return 422)
2. Versicolor prediction (use values: 6.0, 2.7, 4.5, 1.5)
3. Virginica prediction (use values: 7.2, 3.2, 6.0, 1.8)

In [ ]:
# YOUR CODE HERE


---

# Part 8: Deployment

Running the API in production.

## 8.1 Production Server

### Question 8.1: Create Requirements File

In [ ]:
# Create requirements.txt
requirements = """fastapi==0.104.1
uvicorn[standard]==0.24.0
pydantic==2.5.0
scikit-learn==1.3.2
joblib==1.3.2
pytest==7.4.3
httpx==0.25.1
"""

with open("requirements.txt", "w") as f:
    f.write(requirements)

print("Created requirements.txt")
print("\nTo install: pip install -r requirements.txt")

### Question 8.2: Create Dockerfile (Optional)

In [ ]:
# YOUR CODE HERE
# Create a Dockerfile for containerizing the API

dockerfile = """
# YOUR DOCKERFILE HERE
"""

# Hint: Use python:3.10-slim as base image

---

# Summary

In this lab, you learned:

1. **REST API fundamentals**: HTTP methods, status codes, JSON
2. **FastAPI basics**: Routes, path/query parameters
3. **Pydantic validation**: Input/output schemas with constraints
4. **ML model serving**: Loading models, making predictions
5. **Error handling**: Logging, appropriate status codes
6. **Testing**: Using TestClient for API tests
7. **Deployment**: Requirements, production server

## Next Week

**Week 11: Git & CI/CD**

We'll learn to:
- Automate testing with GitHub Actions
- Set up CI/CD pipelines
- Deploy APIs automatically

---

## Submission

Submit:
1. This completed notebook
2. Your `ml_api.py` file
3. Your `test_api.py` file
4. Screenshot of Swagger UI (/docs page)